In [6]:
using Plots
# using StaticArrays
import LinearAlgebra
# import FastGaussQuadrature

push!(LOAD_PATH, pwd())
include("./physconsts.jl")
using .PhysConst


In [ ]:
# module RTtypes
    import Base.@kwdef

    export SGridPar,RGridPar,LongRay, IX,IY,IZ,BIGNUMBER, Rsc
    
    const IX = 1
    const IY = 2
    const IZ = 3
    const BIGNUMBER = 1.0E50
    const Rsc = 1.0 #PhysConst.KM

    @kwdef struct SGridPar
                    
        Nactive = 20;

        nghost = 1 #N of ghost cells at every boundary.
            
        Nx = Nactive;   # N of active cells
        Ny = Nactive;
        Nz = Nactive;

        Nxtot = Nx +  2*nghost;
        Nytot = Ny +  2*nghost;
        Nztot = Nz +  2*nghost;

        is = 1 + nghost; 
        ie = Nxtot -nghost;            
        js = 1 + nghost;  
        je = Nytot - nghost;            
        ks = 1 + nghost;
        ke = Nztot - nghost;

        x1s = -1.0*Rsc
        x1e = 1.0*Rsc        
        x2s = -2.0*Rsc
        x2e = 2.0*Rsc    
        x3s = -3.0*Rsc
        x3e = 3.0*Rsc
        
    end

    @kwdef struct RGridPar        
        Nfreq = 15 ; # ν - N bins         
        NumPhi = 5
        NumTheta = 5        
        Nang = NumPhi*NumTheta    
        N_fre_ang = Nfreq*Nang                      
    end


    mutable struct LongRay
        numOfElm::Int64    
        id::Int64 #id in dir array
        ijkOfCellCrossed :: Matrix{Int64}        
        xyzPos :: Matrix{Float64}
        orig::Int64 #the origin of the ray, as lin index
    end        
    

# end #module RTtypes


In [8]:
# include("RTtypes.jl")

abstract type AbstractSGrid end

# using .RTypes


mutable struct CartGrid
    """ Physical space grid from simulation """    
    x1 :: Matrix{Float64}
end

mutable struct CylGrid
    """ Cylindrical space grid from simulation """
    R :: Matrix{Float64}
    z :: Matrix{Float64}
end

mutable struct TGrid
    """ Tangent space grid from simulation """
    
    x1 :: Matrix{Float64}

end

# testRay = LongRay(Nr, 1, Array{Int64}(undef, 3, Nr), Array{Float64}(undef, 3, Nr), posInRay) 

# make TGrid from SGrid

In [11]:
function SolveRadTransfOnGrid()
        
    Jnew::Float64 = 0.0  # J-mean intens

    # posInRay = 1        
    # Nr = NumElemInRayMax        
    # testRay = LongRay(Nr, 1, Array{Int64}(undef, 3, Nr), Array{Float64}(undef, 3, Nr), posInRay) 

#     if (myid()==1)
#         # @show ie,je,ke = is,js,ks =  xyzToijk( 0.0, 0.0, 0.0 )   
#     else
#         # @everywhere ie,je,ke = is,js,ks =  xyzToijk( 0.0, 0.0, 0.0 )   
#     end

    max_iter = 1; 
    iter_λ = 2;

    while iter_λ <= max_iter         
        # @sync @everywhere for k = ks:ke                                  
        for k = ks:ke                
            println("k = ", k, " myid()= ",   myid(), "\n")
            for j = js:je        
                for i = is:ie    
                    
                    ijk_pos_head .= [i,j,k] #remember your origin parall.id                                                                         
                #    println("ijk= $i $j $k   ", "id =",  myid() )       
                #    println("ijk= $ijk_pos_head ", "id =",  myid() )                        
                WalkThroughAngle( ijk_pos_head )                                                  
                end                       
            end                
    end                    
                        
    #         testRay = nothing                    
    iter_λ += 1     
    end    # while lambda_iter 
    println(" ************ done ************** \n")
    
end # SolveRadTransfOnGrid

SolveRadTransfOnGrid()


# println("done at :  ",  Dates.format(now(), "HH:MM:SS"))


 ************ done ************** 

